In [1]:
from copy import deepcopy
import gurobipy as gp
from gurobipy import GRB
import torch
import torch.nn as nn
import numpy as np
import os
import math
import time
import random
import torch.multiprocessing as multiprocessing

from opt_alg.rl_branch_and_CPT_alg import RLBranchAndCuttingPlaneTreeAlgorithm
from rl_alg.model import var_sorter,var_decider,SharedAdam,Critic

# instanceName = '/mnt/nfs_share/public/sribd_mip_library/2017-MIPLIB_collection/50v-10'
instanceName = '50v-10'


mdl1=var_sorter(v_size=6,c_size=3, sample_sizes=[64,128], multi_head=2, natt=2)
mdl1.share_memory()
mdl2=var_decider(v_size=6,c_size=3, sample_sizes=[64,128], multi_head=2, natt=2)
mdl2.share_memory()
mdl3=Critic(input_size=128*3, sample_sizes=[64,64])
mdl3.share_memory()
global_mdl=[mdl1,
            mdl2,
           mdl3, 
            SharedAdam(list(mdl1.parameters())+list(mdl2.parameters())+list(mdl3.parameters()))]



def train1(idx,lck):
    print(f'Adding worker {idx}...')
    bcpt = RLBranchAndCuttingPlaneTreeAlgorithm(instanceName, lock = lck ,maxIteration = 200, OutputFlag = 0, maxBuffer=5, Threads = 1, MIPGap = 0.0, TimeLimit = 300, nodeSelectionMode = 'BBR', branchVariableSelectionMode = 'MFR', cptVariableSelectionMode = 'MFR')
    bcpt.train(20)
    print('Done')

In [3]:
# multiprocessing.set_start_method('spawn')
# lck=multiprocessing.Lock()

# processes=[]
# n_workers=4

# for i in range(n_workers):
#     p = multiprocessing.Process(target=train1,args=(i,lck))
#     processes.append(p)
#     p.start()

# for p in processes:
#     p.join()
bcpt = RLBranchAndCuttingPlaneTreeAlgorithm(instanceName ,maxIteration = 200, OutputFlag = 0, maxBuffer=5, Threads = 1, MIPGap = 0.0, TimeLimit = 300, nodeSelectionMode = 'BBR', branchVariableSelectionMode = 'RL', cptVariableSelectionMode = 'RL')

bcpt.solve(110)

Read MPS format model from file benchmark/50v-10.mps.gz
Reading time = 0.01 seconds
50v-10: 233 rows, 2013 columns, 2745 nonzeros
Checkpoint loaded
This problem has 1647 integer variables and 0 binary variables.
The optimal value of LP relaxation is 2879.0656868536717.
---------------------------------------------------------------------------------------------------------
|  Iter  |    Lower Bound   |   Overall Improvement   |    Upper Bound   |  Iter Time  |  Overall Time  |
---------------------------------------------------------------------------------------------------------
|       1|         2885.2969|                  0.2164 |              inf |      0.8632 |         2.0469 |
|       2|         2893.6969|                  0.5082 |              inf |      1.2024 |         3.2493 |
|       3|         2897.0900|                  0.6260 |              inf |      1.7626 |         5.0120 |
|       4|         2902.1603|                  0.8022 |              inf |      1.8266 |      